<a href="https://colab.research.google.com/github/0x0checo/NLP/blob/main/%E4%B8%AD%E6%96%87%E7%BD%91%E8%B4%AD%E6%83%85%E6%84%9F%E6%95%B0%E6%8D%AE%E5%88%86%E6%9E%90.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers torch

In [ ]:
!pip install --upgrade datasets

In [ ]:
import torch
import numpy as np
import pandas as pd
import os
from datasets import load_dataset
from transformers import (
    BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments,
    DataCollatorWithPadding
)
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

In [ ]:
# 1.加载数据集
dataset = load_dataset("lansinuote/ChnSentiCorp", download_mode="force_redownload")
print(dataset)

In [ ]:
print(dataset['train'][0])

In [ ]:
# 2.加载分词器
model_name = "google-bert/bert-base-chinese"
tokenizer = BertTokenizer.from_pretrained(model_name)

In [ ]:
# 3. 定义数据预处理函数
def preprocess_data(data):
    encoding = tokenizer(
        data['text'],
        truncation=True,
        padding='max_length',
        max_length=128
    )
    return encoding

In [ ]:
# 4.对数据集进行预处理
encoded_dataset = dataset.map(preprocess_data, batched=True)

功能：map 方法遍历数据集中的所有样本（或批量样本），并对每个样本应用用户定义的函数（preprocess_function），以转换、增强或预处理数据。处理后的结果会生成一个新的数据集（encoded_dataset）。

In [ ]:
print(encoded_dataset)

In [ ]:
# 5.转换数据格式（Hugging Face dataset -> PyTorch dataset）
encoded_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'label'])
print(encoded_dataset['train'][0])

encoded_dataset.set_format(type="torch", columns=["input_ids", "attention_mask", "label"]) 是一个关键的预处理步骤，用于将 Hugging Face 数据集的输出格式转换为 PyTorch 张量，并限制返回的列为模型输入所需的字段（input_ids、attention_mask、label）。

set_format 方法用于设置数据集的输出格式，控制数据集在被访问时返回的数据类型和结构。它不会修改数据的底层存储，而是改变数据被提取时的表示形式。

In [ ]:
# 6.加载模型
model = BertForSequenceClassification.from_pretrained(model_name, num_labels=2)

In [ ]:
# 7.定义数据加载方式
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

data_collator = DataCollatorWithPadding(tokenizer=tokenizer) 定义了一个数据整理器，用于在批处理数据时动态填充变长序列（如 input_ids 和 attention_mask），以满足 Transformer 模型的输入要求。它通过与分词器协作，自动处理填充和注意力掩码生成，确保批次张量形状一致，同时优化内存和计算效率。这一步骤是 Hugging Face Trainer 和 PyTorch DataLoader 的关键组成部分，简化了 NLP 任务的数据准备流程。

In [ ]:
# 8.评估函数
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    accuracy = accuracy_score(labels, predictions)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, predictions, average='binary')
    return {'accuracy': accuracy, "precision": precision, 'recall': recall, 'f1_score': f1}

In [ ]:
# 9.设置训练参数
"""
功能：TrainingArguments 是一个参数容器，用于指定训练过程中的超参数、策略和输出设置。
它为 Trainer 类提供配置，控制训练、评估、模型保存、日志记录等行为。
用途：通过设置 TrainingArguments，用户可以自定义训练流程，例如指定训练轮数、批次大小、
优化器参数、评估频率等，适配不同的任务（如文本分类、机器翻译）和计算资源。
"""
training_args = TrainingArguments(
    output_dir="./out/bert_chinese_sentiment",
    eval_strategy="epoch",
    save_strategy="epoch",
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=3,
    weight_decay=0.01,
    logging_dir="./out/logs",
    logging_steps=500,
    load_best_model_at_end=True,
    metric_for_best_model="accuracy"
)

In [ ]:
# 10.定义Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=encoded_dataset['train'],
    eval_dataset=encoded_dataset['test'],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics
)

In [ ]:
os.environ["WANDB_MODE"] = "disabled"

In [ ]:
# 11.启动训练
trainer.train()

In [ ]:
# 12.评估模型
results = trainer.evaluate()
print(results)

In [ ]:
"""13.保存模型
model.save_pretrained("./out/bert_chinese_sentiment")
tokenizer.save_pretrained("./out/bert_chinese_sentiment")
os.listdir('/content/out/bert_chinese_sentiment/')
"""